In [16]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib import ticker

# ---------------------------- basic parameters --------------------------- #
tmp_dir     = Path("./tmp")                # where klom_*.pt live
forget_ids  = range(1, 9)                  # 1 … 8
n_values    = np.linspace(2, 100, 20, dtype=int)
figsize     = (18, 32)
# ------------------------------------------------------------------------- #

# ---------------------------- load everything ---------------------------- #
data = {fid: {"fgt": {}, "ret": {}, "val": {}} for fid in forget_ids}

for n in n_values:
    res = torch.load(tmp_dir / f"klom_{n}.pt", map_location="cpu")
    for fid in forget_ids:
        for cat in ("fgt", "ret", "val"):
            data[fid][cat][n] = np.asarray(res[cat][fid])

# ---------------------------- start plotting ----------------------------- #
plt.style.use("seaborn-v0_8-whitegrid")

n_rows, n_cols = len(forget_ids), 3
fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=figsize,
                         sharex=False, sharey=False,   # every subplot shows ticks
                         constrained_layout=True)

# ⬤ / ◆ marker styles for percentiles
percentiles = {
    95: dict(marker="o", ms=7, mfc="red",   mec="darkred",   label="95 %"),
    99: dict(marker="D", ms=7, mfc="green", mec="darkgreen", label="99 %")
}

for row, fid in enumerate(forget_ids):
    for col, cat in enumerate(("fgt", "ret", "val")):
        ax = axes[row, col]

        # ----------- box-plot data in n-value order -------------------- #
        series = [np.asarray(data[fid][cat][n]) for n in n_values]

        ax.boxplot(series,
                   positions=n_values,
                   patch_artist=True,            # allow face colour
                   widths=1.5,
                   boxprops   =dict(facecolor="#8ecae6", edgecolor="black", linewidth=1),
                   medianprops=dict(color="#023047", linewidth=1.4),
                   whiskerprops=dict(color="black", linewidth=1),
                   capprops   =dict(color="black", linewidth=1),
                   flierprops =dict(marker=".", markersize=2, alpha=0.4),
                   manage_ticks=False)

        # ----------- overlay 95-th & 99-th percentile markers ---------- #
        for p, style in percentiles.items():
            y = [np.percentile(s, p) for s in series]
            kwargs = {k: v for k, v in style.items() if k != "label"}
            ax.plot(n_values, y, linestyle="None", **kwargs)

        # ---------------- axis cosmetics ------------------------------- #
        ax.set_xticks(n_values)
        ax.set_xticklabels(n_values, rotation=45, ha="right", fontsize=9)
        ax.set_yticks(ax.get_yticks())
        ax.tick_params(axis="y", labelsize=9)
        ax.set_xlabel("number of pre-trained models $N$", fontsize=11)
        ax.set_ylabel(r"$\operatorname{KLOM}$", fontsize=11)
        ax.set_title(cat.upper(), fontsize=12, pad=6)
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))

    # -------- row title spanning the three panels --------------------- #
    axes[row, 1].annotate(f"Forget Set {fid}",
                          xy=(0.5, 1.12), xycoords="axes fraction",
                          ha="center", va="bottom",
                          fontsize=13, fontweight="bold")

# ------------------------- single shared legend -------------------------- #
handles, labels = [], []
for p, style in percentiles.items():
    h, = axes[0, 0].plot([], [], **{k: v for k, v in style.items() if k != "label"})
    handles.append(h); labels.append(style["label"])
fig.legend(handles, labels,
           loc="upper right",
           bbox_to_anchor=(0.99, 1.01),
           frameon=True,
           fontsize=13,
           edgecolor='black',
           fancybox=False)

fig.suptitle("KLOM distribution vs. number of models $N$",
             fontsize=16, y=1.02, fontweight="bold")

plt.savefig("klom_distribution.png")
plt.show()